In [ ]:
#!pip install tqdm

In [6]:
from tqdm import tqdm
import melee
import os
import glob
import torch
import pickle
import json
import multiprocessing as mp

In [7]:
def make_obs(gamestate, max_projectiles=5):
    # 1) Player features extractor (17 floats each)
    def player_feats(p):
        return [
            float(p.stock),
            float(p.percent),
            p.position.x,
            p.position.y,
            float(p.character.value),
            float(p.action.value),
            float(p.action_frame),
            float(p.facing),
            float(p.shield_strength),
            float(p.jumps_left),
            float(p.on_ground),
            float(p.invulnerable),
            p.speed_air_x_self,
            p.speed_ground_x_self,
            p.speed_x_attack,
            p.speed_y_attack,
            p.speed_y_self,
        ]

    # get player slots 1 & 2
    ps = gamestate.players
    f1 = player_feats(ps.get(1)) if 1 in ps else [0.0]*17
    #f2 = player_feats(ps.get(2)) if 2 in ps else [0.0]*17 #we dont want player2 data right?
    f2 = [float(gamestate.players.get(2).character.value)]

    # 2) Stage as one float
    stage_feat = [float(gamestate.stage.value)]

    # 3) Projectile features: each is 7 floats
    proj_feats = []
    for proj in gamestate.projectiles[:max_projectiles]:
        proj_feats.extend([
            float(proj.type.value),
            float(proj.frame),
            float(proj.owner),
            proj.position.x,
            proj.position.y,
            proj.speed.x,
            proj.speed.y,
        ])
    # pad out to max_projectiles * 7
    needed = max_projectiles*7 - len(proj_feats)
    if needed > 0:
        proj_feats.extend([0.0]*needed)

    all_feats = f1 + stage_feat + proj_feats + f2
    return torch.tensor(all_feats, dtype=torch.float32)

In [8]:
# REPLACE WITH YOUR PATH NAME
# source_path = "replays"
source_path = "data"
# find all slp files in the data folder recursively
slp_paths = glob.glob(os.path.join(source_path, "**/*.slp"), recursive=True)
print(len(slp_paths))

output_path = "data/processed"

450


In [9]:
def process_slp(slp):
    states = []
    actions = []
    game_id = os.path.basename(slp).split(".")[0]
    console = melee.Console(system="file", allow_old_version=False, path=slp)
    # console = melee.Console(system="file", path=slp)
    console.connect()
    while True:
        gs = console.step()
        if gs is None:
            break

        controller_state = gs.players[1].controller_state  # or whichever field holds the input
        act = torch.tensor([
            float(controller_state.button[melee.enums.Button.BUTTON_A]),
            float(controller_state.button[melee.enums.Button.BUTTON_B]),
            float(controller_state.button[melee.enums.Button.BUTTON_D_DOWN]),
            float(controller_state.button[melee.enums.Button.BUTTON_D_LEFT]),
            float(controller_state.button[melee.enums.Button.BUTTON_D_RIGHT]),
            float(controller_state.button[melee.enums.Button.BUTTON_D_UP]),
            float(controller_state.button[melee.enums.Button.BUTTON_L]),
            float(controller_state.button[melee.enums.Button.BUTTON_R]),
            float(controller_state.button[melee.enums.Button.BUTTON_X]),
            float(controller_state.button[melee.enums.Button.BUTTON_Y]),
            float(controller_state.button[melee.enums.Button.BUTTON_Z]),
            # float(controller_state.button[melee.enums.Button.BUTTON_START]), # do we need this? @tony
            controller_state.main_stick[0], #x/y components
            controller_state.main_stick[1],
            controller_state.c_stick[0],
            controller_state.c_stick[1],
            controller_state.l_shoulder,
            controller_state.r_shoulder
        ], dtype=torch.float32)

        obs = make_obs(gs, max_projectiles=5)
        states.append(obs)
        actions.append(act)

    # store the states and actions in a pkl file
    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, f"{game_id}.pkl"), "wb") as f:
        pickle.dump({"states": states, "actions": actions}, f)

    return len(actions)

In [10]:
# use multiprocessing to speed up the process
print(f'working with {mp.cpu_count()} cores')
with mp.Pool(processes=mp.cpu_count()) as pool:
    results = pool.map(process_slp, slp_paths)
    
metadata = {}
for k, v in zip(slp_paths, results):
    metadata[os.path.basename(k)] = v

with open(os.path.join(output_path, "metadata.json"), "w") as f:
    json.dump(metadata, f)


working with 12 cores


/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
/home/summertony717/School/cs2

OSError: [Errno 28] No space left on device